In [ ]:
#  https://github.com/veeps/shelter_status

In [1]:
import datetime as dt
import pandas as pd
import re
import numpy as np
import nltk
import regex as reg
import matplotlib.pyplot as plt

from twitterscraper import query_tweets
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from bs4 import BeautifulSoup



pd.set_option('display.max_columns', 500)

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13'}


## DATA COLLECTION

## KINCADE FIRE

In [2]:
lang= 'english'
kincade_begin_date = dt.date(2019,10,23)
kincade_end_date = dt.date(2019,11,15)
limit = 30000

In [4]:
tweets_kincade = query_tweets('kincadefire', begindate=kincade_begin_date, kincade_end_date=end_date,
                     limit=limit, lang=lang)  # October 23 - November 15, 2019

INFO: queries: ['kincadefire since:2019-10-23 until:2019-10-24', 'kincadefire since:2019-10-24 until:2019-10-25', 'kincadefire since:2019-10-25 until:2019-10-26', 'kincadefire since:2019-10-26 until:2019-10-27', 'kincadefire since:2019-10-27 until:2019-10-28', 'kincadefire since:2019-10-28 until:2019-10-29', 'kincadefire since:2019-10-29 until:2019-10-31', 'kincadefire since:2019-10-31 until:2019-11-01', 'kincadefire since:2019-11-01 until:2019-11-02', 'kincadefire since:2019-11-02 until:2019-11-03', 'kincadefire since:2019-11-03 until:2019-11-04', 'kincadefire since:2019-11-04 until:2019-11-05', 'kincadefire since:2019-11-05 until:2019-11-06', 'kincadefire since:2019-11-06 until:2019-11-08', 'kincadefire since:2019-11-08 until:2019-11-09', 'kincadefire since:2019-11-09 until:2019-11-10', 'kincadefire since:2019-11-10 until:2019-11-11', 'kincadefire since:2019-11-11 until:2019-11-12', 'kincadefire since:2019-11-12 until:2019-11-13', 'kincadefire since:2019-11-13 until:2019-11-15']
INFO

INFO: Using proxy 12.218.209.130:53281
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1193940276743745542-1194032896010358784&q=kincadefire%20since%3A2019-11-11%20until%3A2019-11-12&l=english
INFO: Using proxy 12.218.209.130:53281
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1194302253626253313-1194395000769011712&q=kincadefire%20since%3A2019-11-12%20until%3A2019-11-13&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1188967436118192133-1188968668509589504&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 12.218.209.13

INFO: Using proxy 110.5.100.130:60996
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKD9v__zgyEWgMCn7Yva9oMhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-31%20until%3A2019-11-01&l=english
INFO: Using proxy 110.5.100.130:60996
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKb5_KHhjCEWgICnodCn-IwhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-06%20until%3A2019-11-08&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKCNneuRjyEWgMC6yaOFu48hEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-09%20until%3A2019-11-10&l=english
INFO: 

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaMwL7t1q7i_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 117.103.5.186:44825
INFO: Using proxy 117.103.5.186:44825
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKiJhpq3jiEWgMC6yaOFu48hEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-09%20until%3A2019-11-10&l=english
INFO: Using proxy 117.103.5.186:44825
INFO: Retrying... (Attempts left: 41)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=kincadefire%20since%3A2019-10-23%20until%3A2019-10-24&l=english
INFO: Using proxy 193.110.115.220:3128
INFO: Scraping tweets from https://twitter.com/i/se

INFO: Using proxy 190.5.225.178:53570
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgLulicOFiiEWgsCivZ6TroohEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-05%20until%3A2019-11-06&l=english
INFO: Using proxy 190.5.225.178:53570
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLS1q_jugyEWgMCn7Yva9oMhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-31%20until%3A2019-11-01&l=english
INFO: Using proxy 190.5.225.178:53570
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgLe91avJjCEWgICnodCn-IwhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-06

...

INFO: Retrying... (Attempts left: 6)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKeJ4pPT_yAWgMCq-YutiIAhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 92.86.10.42:42658
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwKrZ8Oe5-iAWgICmyY6O9fogEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-24%20until%3A2019-10-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 

INFO: Retrying... (Attempts left: 5)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKeJ4pPT_yAWgMCq-YutiIAhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 183.87.14.250:44915
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwKrZ8Oe5-iAWgICmyY6O9fogEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-24%20until%3A2019-10-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", lin

INFO: Retrying... (Attempts left: 4)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKeJ4pPT_yAWgMCq-YutiIAhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 83.174.233.94:42323
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwKrZ8Oe5-iAWgICmyY6O9fogEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-24%20until%3A2019-10-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", lin

INFO: Retrying... (Attempts left: 3)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKeJ4pPT_yAWgMCq-YutiIAhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 176.119.134.14:23500
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwKrZ8Oe5-iAWgICmyY6O9fogEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-24%20until%3A2019-10-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", li

INFO: Retrying... (Attempts left: 1)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKeJ4pPT_yAWgMCq-YutiIAhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-28%20until%3A2019-10-29&l=english
INFO: Using proxy 181.52.237.106:58484
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgLu9jda5_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", li

In [77]:
df_kincade5=pd.DataFrame(t.__dict__ for t in tweets_kincade)

In [92]:
len(df_kincade5.columns)

21

In [80]:
df_kincade5['hashtags'] = df_kincade5['hashtags'].astype(str)
df_kincade5['hashtags'] = df_kincade5.hashtags.str.replace('[^a-zA-Z0-9]','')

In [81]:
df_kincade5['img_urls'] = df_kincade5['img_urls'].astype(str)
df_kincade5['img_urls'] = df_kincade5['img_urls'].str.strip('[]').astype(str)

In [82]:
df_kincade5['links'] = df_kincade5['links'].astype(str)
df_kincade5['links'] = df_kincade5['links'].str.strip('[]').astype(str)

In [97]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data/Fire'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data/Fire


In [102]:
tweets_kincadefire =df_kincade5

In [103]:
tweets_kincadefire.to_csv('../tweets_kincadefire.csv')

In [104]:
df_kincade5['hashtags'].value_counts().head(3)

KincadeFire    4977
kincadefire     686
Kincadefire     177
Name: hashtags, dtype: int64

## CARR FIRE

In [120]:
carr_begin_date = dt.date(2018,7,23)
carr_end_date = dt.date(2018,9,7)  # Jul 23, 2018 – Aug 30, 2018 add one week - September 7, 2018

In [130]:
tweets_carr_fire = query_tweets('carr fire', begindate=carr_begin_date, enddate=carr_end_date,
                     limit=limit, lang=lang)

INFO: queries: ['carr fire since:2018-07-23 until:2018-07-25', 'carr fire since:2018-07-25 until:2018-07-27', 'carr fire since:2018-07-27 until:2018-07-29', 'carr fire since:2018-07-29 until:2018-08-01', 'carr fire since:2018-08-01 until:2018-08-03', 'carr fire since:2018-08-03 until:2018-08-05', 'carr fire since:2018-08-05 until:2018-08-08', 'carr fire since:2018-08-08 until:2018-08-10', 'carr fire since:2018-08-10 until:2018-08-12', 'carr fire since:2018-08-12 until:2018-08-15', 'carr fire since:2018-08-15 until:2018-08-17', 'carr fire since:2018-08-17 until:2018-08-19', 'carr fire since:2018-08-19 until:2018-08-21', 'carr fire since:2018-08-21 until:2018-08-24', 'carr fire since:2018-08-24 until:2018-08-26', 'carr fire since:2018-08-26 until:2018-08-28', 'carr fire since:2018-08-28 until:2018-08-31', 'carr fire since:2018-08-31 until:2018-09-02', 'carr fire since:2018-09-02 until:2018-09-04', 'carr fire since:2018-09-04 until:2018-09-07']
INFO: Querying carr fire since:2018-07-23 un

INFO: Using proxy 12.218.209.130:53281
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1035992818831904768-1036040714662998016&q=carr%20fire%20since%3A2018-08-31%20until%3A2018-09-02&l=english
INFO: Using proxy 12.218.209.130:53281
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1024442931736797190-1024444565263007744&q=carr%20fire%20since%3A2018-07-29%20until%3A2018-08-01&l=english
INFO: Using proxy 12.218.209.130:53281
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1025889725297643521-1025893906410602496&q=carr%20fire%20since%3A2018-08-03%20until%3A2018-08-05&l=

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKaBwcqF5xwWgsCi2e-KmOccEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-09-04%20until%3A2018-09-07&l=english
INFO: Using proxy 110.5.100.130:60996
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwK2VyejFwBwWgMCmgcuAycAcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-05%20until%3A2018-08-08&l=english
INFO: Using proxy 110.5.100.130:60996
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKely9Wd0RwWgICmtc6dptEcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-19%20until%3A2018-08-21&l=english
INFO: 

INFO: Using proxy 117.103.5.186:44825
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgLyRiN6-1xwWgMC70bfa3dccEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-24%20until%3A2018-08-26&l=english
INFO: Using proxy 117.103.5.186:44825
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKON3abYsxwWisC25eyJ2bMcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-07-27%20until%3A2018-07-29&l=english
INFO: Using proxy 190.5.225.178:53570
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwLbhwO793RwWgICo4Yijld4cEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-28

INFO: Using proxy 190.5.225.178:53570
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe9_4yEsRwWgICm-a68j7EcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-07-25%20until%3A2018-07-27&l=english
INFO: Using proxy 195.69.221.198:54418
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKbJkb2DzBwWgoC8mav0k8wcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-15%20until%3A2018-08-17&l=english
INFO: Using proxy 195.69.221.198:54418
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwKeVnMCq4hwWgICojf3ap-McEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-09-

...

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKbRjMjYyhwWgoC8mav0k8wcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-15%20until%3A2018-08-17&l=english
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaMgL2tzLidvBwWgIC8zdHE2rwcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-03%20until%3A2018-08-05&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/o

INFO: Using proxy 45.7.238.250:30980
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaIwK3F6NKE0hwWgoCn2e28ktUcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-21%20until%3A2018-08-24&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/

ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKbRjMjYyhwWgoC8mav0k8wcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-15%20until%3A2018-08-17&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 355, in raw_de

INFO: Retrying... (Attempts left: 3)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKjBgJ7vxxwWgMCnzdjMyckcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-12%20until%3A2018-08-15&l=english
INFO: Using proxy 165.0.75.137:8080
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaMgL2tzLidvBwWgIC8zdHE2rwcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-03%20until%3A2018-08-05&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 

INFO: Retrying... (Attempts left: 1)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKjBgJ7vxxwWgMCnzdjMyckcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-12%20until%3A2018-08-15&l=english
INFO: Using proxy 85.198.133.19:43157
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaIwK3F6NKE0hwWgoCn2e28ktUcEjUAFQAlAFUAFQAA&q=carr%20fire%20since%3A2018-08-21%20until%3A2018-08-24&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", lin

In [131]:
df_carr=pd.DataFrame(t.__dict__ for t in tweets_carr_fire)

In [132]:
df_carr['hashtags'] = df_carr['hashtags'].astype(str)
df_carr['hashtags'] = df_carr.hashtags.str.replace('[^a-zA-Z0-9]','')
df_carr['img_urls'] = df_carr['img_urls'].astype(str)
df_carr['img_urls'] = df_carr['img_urls'].str.strip('[]').astype(str)
df_carr['links'] = df_carr['links'].astype(str)
df_carr['links'] = df_carr['links'].str.strip('[]').astype(str)

In [134]:
df_carr.to_csv('../df_carr.csv')

## THOMAS FIRE

In [3]:
thomas_begin_date = dt.date(2017,12,4)
thomas_end_date = dt.date(2018,1,19)  # 12/04/2017 - 01/12/2018 plus one week 1/19/2018

In [4]:
thomas_fire = query_tweets('thomas fire', begindate= thomas_begin_date, enddate= thomas_end_date,
                           limit=limit, lang=lang)

INFO: queries: ['thomas fire since:2017-12-04 until:2017-12-06', 'thomas fire since:2017-12-06 until:2017-12-08', 'thomas fire since:2017-12-08 until:2017-12-10', 'thomas fire since:2017-12-10 until:2017-12-13', 'thomas fire since:2017-12-13 until:2017-12-15', 'thomas fire since:2017-12-15 until:2017-12-17', 'thomas fire since:2017-12-17 until:2017-12-20', 'thomas fire since:2017-12-20 until:2017-12-22', 'thomas fire since:2017-12-22 until:2017-12-24', 'thomas fire since:2017-12-24 until:2017-12-27', 'thomas fire since:2017-12-27 until:2017-12-29', 'thomas fire since:2017-12-29 until:2017-12-31', 'thomas fire since:2017-12-31 until:2018-01-02', 'thomas fire since:2018-01-02 until:2018-01-05', 'thomas fire since:2018-01-05 until:2018-01-07', 'thomas fire since:2018-01-07 until:2018-01-09', 'thomas fire since:2018-01-09 until:2018-01-12', 'thomas fire since:2018-01-12 until:2018-01-14', 'thomas fire since:2018-01-14 until:2018-01-16', 'thomas fire since:2018-01-16 until:2018-01-19']
INFO

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-947953102640041985-947979978444017665&q=thomas%20fire%20since%3A2017-12-31%20until%3A2018-01-02&l=english
INFO: Using proxy 158.255.249.58:34239
INFO: Using proxy 158.255.249.58:34239
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-945795120028442624-945806255276339200&q=thomas%20fire%20since%3A2017-12-24%20until%3A2017-12-27&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-949754100345815040-949790122010058752&q=thomas%20fire%20since%3A2018-01-05%20until%3A2018-01-07&l=english
INFO: Scraping tweets from http

INFO: Using proxy 84.22.59.202:8080
INFO: Using proxy 84.22.59.202:8080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLTx2rKUlxoWgICo_fChlZcaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-17%20until%3A2017-12-20&l=english
INFO: Using proxy 84.22.59.202:8080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgK-VhbiepRoWgICh3YqjqaUaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-29%20until%3A2017-12-31&l=english
INFO: Using proxy 84.22.59.202:8080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgK-54fSjtxoWgICi5bjcrLcaEjUAFQAlAFUAFQAA&

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLupnumjihoWjICu4Yy5pYoaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-08%20until%3A2017-12-10&l=english
INFO: Using proxy 103.235.199.46:31611
INFO: Using proxy 188.134.16.191:43665
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwLW5squQrhoWgICtma7Fqq4aEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2018-01-05%20until%3A2018-01-07&l=english
INFO: Using proxy 188.134.16.191:43665
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEgL_drKDbhxoWgsC86ent24caEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2

INFO: Using proxy 103.235.199.46:31611
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKKBnLm6uRoWgMCpgY269bkaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2018-01-14%20until%3A2018-01-16&l=english
INFO: Using proxy 103.235.199.46:31611
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgLeR4erQohoWhsC1ye373qIaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-27%20until%3A2017-12-29&l=english
INFO: Using proxy 109.110.73.106:32479
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaKwLblvNifnBoWgoC3if-0qJwaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2

...

INFO: Got 729 tweets for thomas%20fire%20since%3A2017-12-04%20until%3A2017-12-06.
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKGd_vyvmRoWgICtpfHP3pkaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-20%20until%3A2017-12-22&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 

INFO: Retrying... (Attempts left: 1)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwKGRyYyAjhoWgoCwsbfmk44aEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-10%20until%3A2017-12-13&l=english
INFO: Using proxy 91.187.116.232:44981
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLuVqPuJoxoWgICh3YqjqaUaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-29%20until%3A2017-12-31&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py"

INFO: Got 768 tweets for thomas%20fire%20since%3A2017-12-15%20until%3A2017-12-17.
INFO: Got 712 tweets for thomas%20fire%20since%3A2017-12-06%20until%3A2017-12-08.
INFO: Got 8624 tweets (768 new).
INFO: Got 9336 tweets (712 new).
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe1vqGDoRoWhsC1ye373qIaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-27%20until%3A2017-12-29&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    retur

INFO: Retrying... (Attempts left: 2)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKGd_vyvmRoWgICtpfHP3pkaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-20%20until%3A2017-12-22&l=english
INFO: Using proxy 91.187.116.232:44981
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe1vqGDoRoWhsC1ye373qIaEjUAFQAlAFUAFQAA&q=thomas%20fire%20since%3A2017-12-27%20until%3A2017-12-29&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py"

INFO: Got 785 tweets for thomas%20fire%20since%3A2017-12-20%20until%3A2017-12-22.
INFO: Got 14031 tweets (785 new).


In [5]:
df_thomas=pd.DataFrame(t.__dict__ for t in thomas_fire)

In [8]:
df_thomas['hashtags'] = df_thomas['hashtags'].astype(str)
df_thomas['hashtags'] = df_thomas.hashtags.str.replace('[^a-zA-Z0-9]','')
df_thomas['img_urls'] = df_thomas['img_urls'].astype(str)
df_thomas['img_urls'] = df_thomas['img_urls'].str.strip('[]').astype(str)
df_thomas['links'] = df_thomas['links'].astype(str)
df_thomas['links'] = df_thomas['links'].str.strip('[]').astype(str)

In [10]:
df_thomas.to_csv('../df_thomas.csv')

## COMBINE KINCADE, CARR, THOMAS FIRE DATAFRAMES

In [21]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

In [14]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [18]:
kincade = pd.read_csv('../data/tweets_kincadefire.csv')

In [19]:
carr = pd.read_csv('../data/df_carr.csv')

In [22]:
thomas = pd.read_csv('../data/df_thomas.csv')

In [23]:
kincade.shape

(12676, 22)

In [24]:
carr.shape

(13674, 22)

In [25]:
thomas.shape

(14032, 22)

In [26]:
features = [kincade, carr, thomas]

In [27]:
all_fires =pd.concat(features)

In [29]:
all_fires.shape

(40382, 22)

In [31]:
all_fires.to_csv('../data/all_fires.csv')

## FIRES ALL EXPLORATORY DATA ANALYSIS

In [41]:
floods = pd.read_csv('../data/df_floods.csv')

In [44]:
floods.head(1)

,Unnamed: 0,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,disaster
0,0,ThomasKBullock2,Thomas K. Bullock,813218221603966976,1091479641859198976,/ThomasKBullock2/status/1091479641859198976,2019-02-01 23:33:19,1549063999,"Lord, I hope not. We don’t need another 500 ye...","<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],NaN,1,0,1,True,True,1.091472e+18,"[{'screen_name': 'antmasiello', 'user_id': '54...",floods


In [33]:
firefire = pd.read_csv('../data/all_fires.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,13,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
disaster = 'fire'

In [48]:
firefire['disaster']= disaster

In [37]:
# df.drop(['B', 'C'], axis=1)
firefire= firefire.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [50]:
firefire.to_csv('../data/all_fires.csv')

## DATA CLEANING

In [2]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [3]:
df = pd.read_csv('../data/all_tweets_clean.csv')

In [5]:
df.isnull().sum()

text         0
disaster     3
languages    0
dtype: int64

In [7]:
df.to_csv('../data/df_brian.csv')

In [48]:
X=disaster['text']
y=disaster['disaster']

In [49]:
cv = CountVectorizer(stop_words='english')

In [50]:
cv.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [51]:
disaster_cv = cv.transform(X)

In [52]:
disaster_cv

<67804x81868 sparse matrix of type '<class 'numpy.int64'>'
	with 658388 stored elements in Compressed Sparse Row format>

In [53]:
df_X = pd.DataFrame(disaster_cv.toarray(),
                          columns=cv.get_feature_names())

In [55]:
df_X.shape

(67804, 81868)

In [128]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data/Fire'

In [41]:
# df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv')
df_kincade2.to_csv('../Data/kincade.csv')

In [27]:
#df[df['model'].str.contains('ac')]
#df.loc[df['username'] == some_value]
df_kincade2[df_kincade['username'].str.contains('shelter')]

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
